# PeDistance Linear Regression

In [14]:
import torch
from torch import nn
import pandas as pd

0      13.659012
1      19.537920
2       6.014757
3      12.382139
4      19.352587
         ...    
145     6.513113
146     6.634655
147    15.349739
148    18.069414
149    17.966396
Name: Distance, Length: 150, dtype: float64
0       [62.0, 187.0]
1       [65.0, 138.0]
2      [159.0, 420.0]
3       [97.0, 203.0]
4       [49.0, 121.0]
            ...      
145    [155.0, 395.0]
146    [157.0, 380.0]
147     [50.0, 150.0]
148     [55.0, 131.0]
149     [52.0, 129.0]
Length: 150, dtype: object


### Import Dataset

In [ ]:
fields = [
    'Distance', 
    'PedTopLeftX', 
    'PedTopLeftY', 
    'PedTopRightX', 
    'PedTopRightY',
    'PedBottomLeftX', 
    'PedBottomLeftY',
    'PedBottomRightX',
    'PedBottomRightY'
]

df = pd.read_csv (r'./Dataset.csv', usecols=fields)

In [15]:
print(df)

      Distance  PedTopLeftX  PedTopLeftY  PedTopRightX  PedTopRightY  \
0    13.659012         1480          519          1542           519   
1    19.537920          382          625           447           625   
2     6.014757          796          558           955           558   
3    12.382139         1408          481          1505           481   
4    19.352587          884          543           933           543   
..         ...          ...          ...           ...           ...   
145   6.513113          532          524           687           524   
146   6.634655          493          528           650           528   
147  15.349739         1049          518          1099           518   
148  18.069414          711          508           766           508   
149  17.966396          937          511           989           511   

     PedBottomLeftX  PedBottomLeftY  PedBottomRightX  PedBottomRightY  
0              1480             706             1542           

### Parsing raw dataset data

In [ ]:
target = df.Distance
print(Distance)

In [16]:
data = df.apply(lambda el : [abs(el.PedTopRightX - el.PedTopLeftX), abs(el.PedBottomLeftY - el.PedTopLeftY)], axis=1)
print(champ)

0       [62.0, 187.0]
1       [65.0, 138.0]
2      [159.0, 420.0]
3       [97.0, 203.0]
4       [49.0, 121.0]
            ...      
145    [155.0, 395.0]
146    [157.0, 380.0]
147     [50.0, 150.0]
148     [55.0, 131.0]
149     [52.0, 129.0]
Length: 150, dtype: object


In [ ]:
class LinearRegressor(nn.Module):
    def __init__(self, in_size, out_size):
        super(LinearRegressor, self).__init__()
        self.linear = nn.Linear(in_size,out_size)
    
    def forward(self,x):
        result = self.linear(x)
        return result

In [ ]:
lr = 0.001
#facciamo training per 5000 epoche in modo da verificare che
#tensorboard viene aggiornato in tempo reale
epochs = 5000

writer = SummaryWriter('logs/linear_regressor_lr0001');

#normalizzazione dei dati
means = X_training.mean(0)
stds = X_training.std(0)

X_training_norm = (X_training-means)/stds
X_testing_norm = (X_testing-means)/stds

reg = LinearRegressor(13,1)
criterion = nn.MSELoss()

optimizer = torch.optim.SGD(reg.parameters(),lr=lr)

for e in range(epochs):
    reg.train()
    output = reg(X_training_norm)
    l = criterion(output.view(-1),Y_training)

    writer.add_scalar('loss/train', l.item(), global_step=e)
    
    l.backward()
    
    optimizer.step()    
    optimizer.zero_grad()
    
    reg.eval()
 
    with torch.set_grad_enabled(False):
        y_test = reg(X_testing_norm)
        l = criterion(y_test.view(-1),Y_testing)
        writer.add_scalar('loss/test', l.item(), global_step=e)